<a href="https://colab.research.google.com/github/DIPANJAN001/Dipanjanpatra/blob/master/Report2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install Boruta
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import concatenate
from keras import Model
from keras.layers import Input
from keras.layers import Bidirectional
from keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.layers import Input
from sklearn.decomposition import PCA 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
df=pd.read_excel("/content/pv_10.xlsx")
weather_input1=df.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
solpow=df['power_normed']

In [3]:
def lstm_data_transform(x_data, y_data, num_steps):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [4]:
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.layers import Input

In [5]:
n_inputs=weather_input.shape[1]

In [6]:
input_data_shape= Input(shape=(n_inputs,))
# encoder level 1
encoder= Dense(n_inputs*2)(input_data_shape)
encoder = BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# encoder level 2
encoder= Dense(n_inputs)(encoder)
encoder= BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# bottleneck
#n_bottleneck = round(float(n_inputs) / 2.0)
n_bottleneck = 20
bottleneck = Dense(n_bottleneck)(encoder)
# define decoder, level 1
decoder = Dense(n_inputs)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)
# decoder level 2
decoder = Dense(n_inputs*2)(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)

In [7]:
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(optimizer='adam', loss='mse')

In [8]:
 model.fit(weather_input,weather_input, epochs=100, batch_size=32, verbose=2)

Epoch 1/100
192/192 - 2s - loss: 0.0951 - 2s/epoch - 11ms/step
Epoch 2/100
192/192 - 1s - loss: 0.0195 - 508ms/epoch - 3ms/step
Epoch 3/100
192/192 - 1s - loss: 0.0155 - 529ms/epoch - 3ms/step
Epoch 4/100
192/192 - 0s - loss: 0.0133 - 474ms/epoch - 2ms/step
Epoch 5/100
192/192 - 1s - loss: 0.0120 - 669ms/epoch - 3ms/step
Epoch 6/100
192/192 - 1s - loss: 0.0111 - 510ms/epoch - 3ms/step
Epoch 7/100
192/192 - 1s - loss: 0.0104 - 504ms/epoch - 3ms/step
Epoch 8/100
192/192 - 0s - loss: 0.0101 - 489ms/epoch - 3ms/step
Epoch 9/100
192/192 - 0s - loss: 0.0099 - 483ms/epoch - 3ms/step
Epoch 10/100
192/192 - 0s - loss: 0.0093 - 480ms/epoch - 3ms/step
Epoch 11/100
192/192 - 1s - loss: 0.0090 - 520ms/epoch - 3ms/step
Epoch 12/100
192/192 - 1s - loss: 0.0086 - 500ms/epoch - 3ms/step
Epoch 13/100
192/192 - 1s - loss: 0.0083 - 1s/epoch - 5ms/step
Epoch 14/100
192/192 - 2s - loss: 0.0084 - 2s/epoch - 9ms/step
Epoch 15/100
192/192 - 2s - loss: 0.0077 - 2s/epoch - 12ms/step
Epoch 16/100
192/192 - 1s - l

In [9]:
# define an encoder model (without the decoder)
encoder = Model(inputs=input_data_shape, outputs=bottleneck)
# save the encoder to file
encoder.save('encoder.h5')

In [10]:
#solpow=solpow.to_numpy().reshape(-1,1)
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(weather_input)

192/192 [==============================] - 0s 1ms/step


In [11]:
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(weather_input)

192/192 [==============================] - 0s 1ms/step


In [12]:
num_steps = 3
# training set
(x_transformed1,
 y_transformed1) = lstm_data_transform(X_train_encode,solpow , num_steps=num_steps)
assert x_transformed1.shape[0] == y_transformed1.shape[0]

In [13]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(x_transformed1, y_transformed1, test_size=0.4, random_state=42,shuffle=False)

In [14]:
inputs1=Input(shape=(X_train1.shape[1],X_train1.shape[2]))

In [15]:
def get_model1(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe1_0 = Bidirectional(LSTM(32, activation='relu',return_sequences = True))(inputs1)#32
    fe1_1 = Dropout(0.2)(fe1_0)
    fe1_2 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(fe1_1)#16
    fe1_3= Dropout(0.2)(fe1_2)
    fe1_4=Bidirectional(LSTM(8, activation='relu'))(fe1_3)#8
    out1_1=Dense(1, activation='linear')(fe1_4)
    return Model(inputs1, out1_1)
def get_model2(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe2_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)#16,16,8
    fe2_1 = Dropout(0.5)(fe2_0)
    fe2_2 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(fe2_1)
    fe2_3= Dropout(0.5)(fe2_2)
    fe2_4=Bidirectional(LSTM(8, activation='relu'))(fe2_3)
    out2_1=Dense(1, activation='linear')(fe2_4)
    return Model(inputs1, out2_1)
def get_model3(): 
    #inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))
    fe3_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)#16,8,8
    fe3_1 = Dropout(0.5)(fe3_0)
    fe3_2 = Bidirectional(LSTM(8, activation='relu',return_sequences = True))(fe3_1)
    fe3_3= Dropout(0.5)(fe3_2)
    fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_3)
    out3_1=Dense(1, activation='linear')(fe3_4)
    return Model(inputs1, out3_1)
model1 = get_model1() 
model2 = get_model2() 
model3 = get_model3()
y1 = model1(inputs1) 
y2 = model2(inputs1) 
y3 = model3(inputs1)
outputs = layers.average([y1, y2, y3]) 
ensemble_model = Model(inputs=inputs1, outputs=outputs)
ensemble_model.compile(optimizer='Adam',loss='mean_squared_error',metrics=['RootMeanSquaredError'])
history=ensemble_model.fit(X_train1, y_train1, epochs = 200,validation_split=0.2,batch_size = 32)

Epoch 1/200
93/93 [==============================] - 34s 76ms/step - loss: 0.0125 - root_mean_squared_error: 0.1116 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0392
Epoch 2/200
93/93 [==============================] - 3s 30ms/step - loss: 0.0049 - root_mean_squared_error: 0.0699 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0371
Epoch 3/200
93/93 [==============================] - 3s 29ms/step - loss: 0.0040 - root_mean_squared_error: 0.0631 - val_loss: 0.0012 - val_root_mean_squared_error: 0.0342
Epoch 4/200
93/93 [==============================] - 3s 30ms/step - loss: 0.0036 - root_mean_squared_error: 0.0598 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0331
Epoch 5/200
93/93 [==============================] - 3s 29ms/step - loss: 0.0032 - root_mean_squared_error: 0.0567 - val_loss: 0.0010 - val_root_mean_squared_error: 0.0318
Epoch 6/200
93/93 [==============================] - 3s 27ms/step - loss: 0.0034 - root_mean_squared_error: 0.0579 - val_loss: 0.0011 - val

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-4544448dcf39>", line 37, in <module>
    history=ensemble_model.fit(X_train1, y_train1, epochs = 200,validation_split=0.2,batch_size = 32)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1409, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 915, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eag

KeyboardInterrupt: ignored

In [ ]:
y_en=ensemble_model.predict(X_test1)
plt.scatter(y_en,y_test1)
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
df_ = pd.DataFrame()
df_['time']=[i for i in range(len(y_en))]
df_['Actual']=y_test1
df_['Predicted']=y_en

In [ ]:
print(np.sqrt(mean_squared_error(y_en,y_test1)))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df_['time'],df_['Actual'])
plt.plot(df_['time'],df_['Predicted'])
plt.show()

In [ ]:
inputs1=Input(shape=(X_train1.shape[1],X_train1.shape[2]))

In [ ]:
def define_model(hp):


  # firs
  fe1_0 = LSTM(hp.Int('input_unit',min_value=16,max_value=32,step=2), activation='relu',return_sequences = True)(inputs1)
  fe1_1 = Dropout(0.3)(fe1_0)
  #fe1_2 = LSTM(16, activation='relu',return_sequences = True)(fe1_1)
  #fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=LSTM(hp.Int('input_unit',min_value=8,max_value=32,step=2), activation='relu')(fe1_1)
  out1_1=Dense(1, activation='linear')(fe1_4)



  fe2_0 = LSTM(hp.Int('input_unit',min_value=16,max_value=64,step=4), activation='relu',return_sequences = True)(inputs1)
  fe2_1 = Dropout(0.3)(fe2_0)
  #fe2_2 = LSTM(16, activation='relu',return_sequences = True)(fe2_1)
  #fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=LSTM(hp.Int('input_unit',min_value=4,max_value=32,step=4), activation='relu')(fe2_1)
  out2_1=Dense(1, activation='linear')(fe2_4)

  fe3_0 = LSTM(hp.Int('input_unit',min_value=8,max_value=32,step=8), activation='relu',return_sequences = True)(inputs1)
  fe3_1 = Dropout(0.3)(fe3_0)
  #fe3_2 = LSTM(8, activation='relu',return_sequences = True)(fe3_1)
  #fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=LSTM(hp.Int('input_unit',min_value=8,max_value=16,step=2), activation='relu')(fe3_1)
  out3_1=Dense(1, activation='linear')(fe3_4)
 
 

  merged3 = concatenate([out1_1,out2_1,out3_1], name='concat3')
  output = Dense(1, activation='linear')( merged3)

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['RootMeanSquaredError'])
 
  return model
mdl = define_model()
history=mdl.fit(X_train1, y_train1, epochs = 200,validation_split=0.2,batch_size = 64)
y1=mdl.predict(X_test1)
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:

y1=mdl.predict(X_test1)
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
def define_model_():


  # firs
  fe1_0 = Bidirectional(LSTM(32, activation='relu',return_sequences = True))(inputs1)
  fe1_1 = Dropout(0.25)(fe1_0)
  #fe1_2 = LSTM(16, activation='relu',return_sequences = True)(fe1_1)
  #fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=Bidirectional(LSTM(8, activation='relu'))(fe1_1)
  out1_1=Dense(1, activation='linear')(fe1_4)



  fe2_0 = Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)
  fe2_1 = Dropout(0.25)(fe2_0)
  #fe2_2 = LSTM(16, activation='relu',return_sequences = True)(fe2_1)
  #fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=Bidirectional(LSTM(8, activation='relu'))(fe2_1)
  out2_1=Dense(1, activation='linear')(fe2_4)

  fe3_0 =Bidirectional(LSTM(16, activation='relu',return_sequences = True))(inputs1)
  fe3_1 = Dropout(0.25)(fe3_0)
  #fe3_2 = LSTM(8, activation='relu',return_sequences = True)(fe3_1)
  #fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_1)
  out3_1=Dense(1, activation='linear')(fe3_4)
 
 

  merged3 = concatenate([out1_1,out2_1,out3_1], name='concat3')
  output = Dense(1, activation='linear')( merged3)

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['RootMeanSquaredError'])
 
  return model
mdl = define_model_()
history=mdl.fit(X_train1, y_train1, epochs = 200,validation_split=0.2,batch_size = 64)
y1=mdl.predict(X_test1)
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
!pip install keras_tuner
import keras_tuner as kt

In [ ]:
'''y1=mdl.predict(X_test1)
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()'''

In [ ]:
def define_model_(hp):


  # firs
  fe1_0 = LSTM(hp.Int('input_unit11',min_value=16,max_value=32,step=2), activation='relu',return_sequences = True)(inputs1)
  fe1_1 = Dropout(0.3)(fe1_0)
  #fe1_2 = LSTM(16, activation='relu',return_sequences = True)(fe1_1)
  #fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=LSTM(hp.Int('input_unit12',min_value=8,max_value=32,step=2), activation='relu')(fe1_1)
  out1_1=Dense(1, activation='linear')(fe1_4)



  fe2_0 = LSTM(hp.Int('input_unit21',min_value=16,max_value=64,step=4), activation='relu',return_sequences = True)(inputs1)
  fe2_1 = Dropout(0.3)(fe2_0)
  #fe2_2 = LSTM(16, activation='relu',return_sequences = True)(fe2_1)
  #fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=LSTM(hp.Int('input_unit22',min_value=4,max_value=32,step=4), activation='relu')(fe2_1)
  out2_1=Dense(1, activation='linear')(fe2_4)

  fe3_0 = LSTM(hp.Int('input_unit31',min_value=8,max_value=32,step=8), activation='relu',return_sequences = True)(inputs1)
  fe3_1 = Dropout(0.3)(fe3_0)
  #fe3_2 = LSTM(8, activation='relu',return_sequences = True)(fe3_1)
  #fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=LSTM(hp.Int('input_unit32',min_value=8,max_value=16,step=2), activation='relu')(fe3_1)
  out3_1=Dense(1, activation='linear')(fe3_4)
 
 

  output = layers.average([ out1_1, out2_1, out2_1])
  

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
 
  return model

In [ ]:
def define_model(hp):


  # firs
  fe1_0 = LSTM(hp.Int('input_unit11',min_value=16,max_value=32,step=2), activation='relu',return_sequences = True)(inputs1)
  fe1_1 = Dropout(0.3)(fe1_0)
  #fe1_2 = LSTM(16, activation='relu',return_sequences = True)(fe1_1)
  #fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=LSTM(hp.Int('input_unit12',min_value=8,max_value=32,step=2), activation='relu')(fe1_1)
  out1_1=Dense(1, activation='linear')(fe1_4)



  fe2_0 = LSTM(hp.Int('input_unit21',min_value=16,max_value=64,step=4), activation='relu',return_sequences = True)(inputs1)
  fe2_1 = Dropout(0.3)(fe2_0)
  #fe2_2 = LSTM(16, activation='relu',return_sequences = True)(fe2_1)
  #fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=LSTM(hp.Int('input_unit22',min_value=4,max_value=32,step=4), activation='relu')(fe2_1)
  out2_1=Dense(1, activation='linear')(fe2_4)

  fe3_0 = LSTM(hp.Int('input_unit31',min_value=8,max_value=32,step=8), activation='relu',return_sequences = True)(inputs1)
  fe3_1 = Dropout(0.3)(fe3_0)
  #fe3_2 = LSTM(8, activation='relu',return_sequences = True)(fe3_1)
  #fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=LSTM(hp.Int('input_unit32',min_value=8,max_value=16,step=2), activation='relu')(fe3_1)
  out3_1=Dense(1, activation='linear')(fe3_4)
 
 

  merged3 = concatenate([out1_1,out2_1,out3_1], name='concat3')
  output = Dense(1, activation='linear')( merged3)

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
 
  return model
tuner1 = kt.RandomSearch(
    define_model,
    objective='mse',
    max_trials=3,
    executions_per_trial=2
    )
tuner1.search(
        x=X_train1,
        y=y_train1,
        epochs=50,
        batch_size=64,
        validation_data=(X_test1,y_test1),
)
best_model1 = tuner1.get_best_models(num_models=1)[0]
y1=best_model1.predict(X_test1)
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
!pip install keras_tuner


In [ ]:
import keras_tuner as kt


In [ ]:
tuner1 = kt.RandomSearch(
    define_model,
    objective='mse',
    max_trials=3,
    executions_per_trial=2
    )

In [ ]:
tuner1.search(
        x=X_train1,
        y=y_train1,
        epochs=50,
        batch_size=64,
        validation_data=(X_test1,y_test1),
)

In [ ]:
best_model1 = tuner1.get_best_models(num_models=1)[0]

In [ ]:
y1=best_model1.predict(X_test1)

In [ ]:
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
def define_model_(hp):


  # firs
  fe1_0 = LSTM(hp.Int('input_unit11',min_value=16,max_value=32,step=2), activation='relu',return_sequences = True)(inputs1)
  fe1_1 = Dropout(hp.Float('Dropout_rate',min_value=0.2,max_value=0.5,step=0.1))(fe1_0)
  #fe1_2 = LSTM(16, activation='relu',return_sequences = True)(fe1_1)
  #fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=LSTM(hp.Int('input_unit12',min_value=8,max_value=32,step=2), activation='relu')(fe1_1)
  out1_1=Dense(1, activation='linear')(fe1_4)



  fe2_0 = LSTM(hp.Int('input_unit21',min_value=16,max_value=64,step=4), activation='relu',return_sequences = True)(inputs1)
  fe2_1 = Dropout(hp.Float('Dropout_rate',min_value=0.2,max_value=0.5,step=0.1))(fe2_0)
  #fe2_2 = LSTM(16, activation='relu',return_sequences = True)(fe2_1)
  #fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=LSTM(hp.Int('input_unit22',min_value=4,max_value=32,step=4), activation='relu')(fe2_1)
  out2_1=Dense(1, activation='linear')(fe2_4)

  fe3_0 = LSTM(hp.Int('input_unit31',min_value=8,max_value=32,step=8), activation='relu',return_sequences = True)(inputs1)
  fe3_1 = Dropout(hp.Float('Dropout_rate',min_value=0.2,max_value=0.5,step=0.1))(fe3_0)
  #fe3_2 = LSTM(8, activation='relu',return_sequences = True)(fe3_1)
  #fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=LSTM(hp.Int('input_unit32',min_value=8,max_value=16,step=2), activation='relu')(fe3_1)
  out3_1=Dense(1, activation='linear')(fe3_4)
 
 

  output = layers.average([ out1_1, out2_1, out2_1])
  

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
 
  return model




In [ ]:
tuner = kt.RandomSearch(
    define_model_,
    objective='mse',
    max_trials=3,
    executions_per_trial=2
    )

In [ ]:
tuner.search(
        x=X_train1,
        y=y_train1,
        epochs=50,
        batch_size=64,
        validation_data=(X_test1,y_test1)
)


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
y1=best_model.predict(X_test1)
print(np.sqrt(mean_squared_error(y1,y_test1)))
plt.scatter(y1,y_test1)
plt.show()
df_2 = pd.DataFrame()
df_2['time']=[i for i in range(len(y1))]
df_2['Actual']=y_test1
df_2['Predicted']=y1
import matplotlib.pyplot as plt
plt.plot(df_2['time'],df_2['Actual'])
plt.plot(df_2['time'],df_2['Predicted'])
plt.show()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()